In [1]:
%matplotlib ipympl

import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import copy
from classes.Geometry import Geometry
from classes.Phonon import Phonon
from classes.Population import Population

d:\LEMTA\Code\MultiscaleThermalCond\classes\Jitted.py:122: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 1d, A), array(float64, 2d, C))
  prob[iif, :, :] = calculate_diffuse_probs_jit(conc_unique[iif, 0:3], conc_unique[iif, 3], q_all, v_all, rl, data_mesh)


In [2]:
class args():
    def __init__(self):
        self.hdf_file       = ['D:\LEMTA\Code\Materials\Si\kappa-m313131.hdf5']
        self.poscar_file    = ['D:\LEMTA\Code\Materials\Si\POSCAR']
        self.mat_names      = ['silicon']
        self.reference_temp = [295]
        self.mat_rotation   = []

args = args()

phonon = Phonon(args, 0)
phonon.load_properties()

Reading hdf file...
Expanding frequency to FBZ...
Expanding group velocity to FBZ...
Expanding gamma to FBZ...
To describe phonons:
 nq= 29791
 nb= 6
 number of modes= 178746
Interpolating lifetime...
Generating T = f(E)...
Generating p_ref = f(T)...
Material initialisation done!


In [3]:
# Importing data

file = 'D:\LEMTA\Code\Results\z_wire_high_grad_0\particle_data_400ps.txt'

sim_data = np.loadtxt(file, delimiter = ',')

sv = 0

ind = np.logical_and(sim_data[:, 2] >= sv*2e3/5, sim_data[:, 2] < (sv+1)*2e3/5)

sim_data = sim_data[ind, :]

sim_mod  = copy.copy(sim_data[:, 0:2])
sim_pos  = copy.copy(sim_data[:, 2:5])
sim_occ  = copy.copy(sim_data[:, 5  ])

del(sim_data)

In [4]:
sim_mod = sim_mod.astype(int)
sim_omega = phonon.omega[sim_mod[:, 0], sim_mod[:, 1]]

g, edges = np.histogram(phonon.omega, bins = 200)
centers  = (edges[:-1]+edges[1:])/2

plt.close('all')
fig, ax = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 5), dpi = 100)

# density of states
ax[0].hist(sim_omega   , bins = edges, density = True)
ax[0].hist(phonon.omega.reshape(-1), bins = edges, density = True, histtype = 'step')
ax[0].set_title(r'Density of states $g(\omega)$')
ax[0].legend(['Expected (from phonon data)', 'Simulation (from particle data)'])

ax[0].set_xlabel(r'$\omega$ [THz $\cdot$ rad]')
ax[0].set_ylabel(r'$g(\omega)$')


# omega x occupation

n = phonon.calculate_occupation(T = 300.72, omega = centers, reference = True)

sim_h = np.zeros(edges.shape[0]-1)
for i in range(edges.shape[0]-1):
    ind = np.logical_and(sim_omega >= edges[i], sim_omega < edges[i+1])
    sim_h[i] = np.sum(sim_occ[ind])


ax[1].hist(centers, bins = edges, weights = sim_h, density = True)
ax[1].hist(centers, bins = edges, weights = g*n, density = True, histtype = 'step')
ax[1].set_title(r'Density of phonons for each frequency $n(\omega)$')
ax[1].legend(['Expected (from phonon data)', 'Simulation (from particle data)'])

ax[1].set_xlabel(r'$\omega$ [THz $\cdot$ rad]')
ax[1].set_ylabel(r'$n(\omega)$')

plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …